In [ ]:
"""
Classifications and Regressions using features generated from the computed mask

"""

In [4]:
# Necessary Library Calls
import nibabel as nib
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from skimage.util import montage 
from skimage.transform import rotate
from sklearn.model_selection import train_test_split as ttt
import torch
import torch.nn as nn 
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
from tqdm import tqdm
from pylab import *
import gc
import torchmetrics 
import radiomics
from torchvision.utils import save_image
import matplotlib.image
import SimpleITK as sitk
import radiomics



In [5]:
# Global variable definition
IMAGE_PATH = 'data/MICCAI_BraTS_2019_Data_Training'
CSV_FILE='data/MICCAI_BraTS_2019_Data_Training/new.csv'
MODEL_PATH = 'log/21/'

# number of slices, meaning how many slices we are taking from one image file  
SLICE_NUM = 10
# Original Image Size
IMAGE_SIZE=240
# Image size after cropping
HEIGHT=128
WIDTH=144

# Batch size constant
BATCH_SIZE= 1
# input_channel corresponds to the number of different types of images used: 
# e.g: T1, T2, T1_flair, etc. 
# only using one type of images at the moment
INPUT_CHANNEL=2
EPOCH = 50
LEARNING_RATE = 0.001
SLICE_STARTS=torch.load('data/slice_starts.pt')
device = torch.device('cuda')
LOAD_CURRENT = False
LOAD_BEST = True


In [6]:
FEAT_PATH ='log/feature_extraction/'
tf=torch.load(FEAT_PATH+'tf.pt')
ef=torch.load(FEAT_PATH+'ef.pt')
ttf=torch.load(FEAT_PATH+'ttf.pt')

ts=torch.load(FEAT_PATH+'ts.pt')
es=torch.load(FEAT_PATH+'es.pt')
tts=torch.load(FEAT_PATH+'tts.pt')

tc=torch.load(FEAT_PATH+'tc.pt')
ec=torch.load(FEAT_PATH+'ec.pt')
ttc=torch.load(FEAT_PATH+'ttc.pt')





In [ ]:
"""

Classificaton: 

Patients are divided into three classes:

short-term: survival days < 10 months
mid-term: 10 months < survival days < 15 months
long-term: survival days > 15 months

"""

In [7]:
from sklearn.svm import SVC
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import BaggingClassifier
from sklearn.svm import LinearSVC

In [ ]:



t1=tf
e1=ef
tt1=ttf
print(e1.shape)
print(ec.cpu().numpy())
clf = SVC(kernel='linear').fit(e1.cpu().numpy(),ec.cpu().numpy())




torch.Size([36, 88])
[2. 1. 1. 0. 1. 1. 0. 2. 2. 0. 1. 1. 2. 0. 1. 1. 0. 1. 0. 0. 1. 1. 0. 0.
 1. 2. 1. 1. 1. 0. 1. 0. 1. 1. 1. 0.]


In [187]:

t1=tf
e1=ef
tt1=ttf
selector = SelectFromModel(estimator=LogisticRegression()).fit(t1, tc)
t1=selector.transform(t1)
e1=selector.transform(e1)
tt1=selector.transform(tt1)

clf=LogisticRegression().fit(t1,tc)
clf.score(e1,ttc)



0.5277777777777778

In [16]:
from sklearn.svm import LinearSVC
t1=tf
e1=ef

tt1=ttf
s = SelectFromModel(estimator=LinearSVC(dual=False)).fit(t1, tc)
t1=s.transform(t1)
e1=s.transform(e1)
tt1=s.transform(tt1)

clf=LinearSVC(dual=False).fit(t1,tc)
clf.score(t1,tc)



0.43636363636363634

In [149]:
selector.get_support()
selector.transform(t1)


array([[1.21208627e+09, 1.21208627e+09, 1.06051859e+09, 1.06051859e+09],
       [1.91398624e+08, 1.91398624e+08, 4.07754848e+08, 4.07754848e+08],
       [6.83496653e+10, 6.83496653e+10, 1.12211354e+10, 1.12211354e+10],
       [3.34279620e+07, 3.34279620e+07, 7.91431840e+07, 7.91431840e+07],
       [2.23592256e+08, 2.23592256e+08, 4.01594752e+08, 4.01594752e+08],
       [1.91421152e+08, 1.91421152e+08, 3.61693824e+08, 3.61693824e+08],
       [5.75114304e+08, 5.75114304e+08, 2.64149376e+08, 2.64149376e+08],
       [3.90097600e+07, 3.90097600e+07, 1.58938800e+07, 1.58938800e+07],
       [3.46162509e+09, 3.46162509e+09, 2.66582963e+09, 2.66582963e+09],
       [3.80605760e+08, 3.80605760e+08, 2.57065024e+08, 2.57065024e+08],
       [6.14488384e+08, 6.14488384e+08, 5.87968832e+08, 5.87968832e+08],
       [6.53892544e+08, 6.53892544e+08, 5.27044448e+08, 5.27044448e+08],
       [8.27587200e+08, 8.27587200e+08, 7.28163136e+08, 7.28163136e+08],
       [1.44158864e+08, 1.44158864e+08, 1.30993048e

In [150]:
clf = LogisticRegression(random_state=0).fit(t1, tc)

In [151]:
clf.score(t1, tc)

0.43636363636363634

In [152]:
from sklearn.ensemble import ExtraTreesClassifier
selector = SelectFromModel(estimator=ExtraTreesClassifier(n_estimators=50)).fit(tf, tc)

In [153]:
t1=tf
selector.get_support()


array([False, False,  True,  True,  True,  True,  True,  True, False,
       False, False,  True, False,  True, False,  True,  True, False,
        True,  True,  True,  True,  True,  True, False,  True,  True,
       False, False, False,  True, False, False, False, False,  True,
        True, False, False, False, False, False, False, False, False,
       False, False, False,  True, False, False, False, False, False,
        True,  True, False, False,  True, False,  True, False,  True,
       False, False,  True, False,  True,  True, False, False, False,
        True,  True,  True, False, False,  True, False, False,  True,
        True, False, False, False, False,  True,  True])

In [154]:
t1=ef
selector.transform(t1)

array([[1.6774779e+03, 4.1133627e-01, 1.3995692e-02, ..., 5.6817696e+01,
        0.0000000e+00, 5.6550686e+01],
       [2.0706106e+03, 3.4537622e-01, 1.0768062e-02, ..., 1.5095553e+01,
        0.0000000e+00, 5.1756165e+01],
       [5.2834517e+03, 5.2794075e-01, 2.0350572e-02, ..., 1.1588735e+01,
        0.0000000e+00, 3.1967123e+01],
       ...,
       [2.0203104e+03, 3.4519389e-01, 1.0759536e-02, ..., 6.5421720e+00,
        1.0000000e+00, 6.5115067e+01],
       [9.4777429e+02, 2.8413337e-01, 8.0349334e-03, ..., 1.5923249e+01,
        1.0000000e+00, 6.0698631e+01],
       [1.1355579e+03, 4.5397761e-01, 1.6227460e-02, ..., 4.7212458e+00,
        1.0000000e+00, 6.0000000e+01]], dtype=float32)

In [155]:
clf = ExtraTreesClassifier(n_estimators=50).fit(t1, tc)

In [ ]:
clf.score(e1,ec)

array([[3.5559312e+03, 3.4892747e-01, 1.0934568e-02, ..., 1.8379179e+01,
        0.0000000e+00, 6.0413700e+01],
       [3.4340444e+03, 4.1096082e-01, 1.3976535e-02, ..., 1.6116812e+02,
        1.0000000e+00, 3.9000000e+01],
       [1.7294396e+03, 4.0873712e-01, 1.3863248e-02, ..., 9.7704079e+01,
        0.0000000e+00, 2.7904110e+01],
       ...,
       [3.3646284e+03, 4.8883533e-01, 1.8131876e-02, ..., 5.4801497e+00,
        0.0000000e+00, 6.0330395e+01],
       [2.1830483e+03, 2.6239780e-01, 7.1308143e-03, ..., 5.2007880e+00,
        0.0000000e+00, 6.0330395e+01],
       [1.9795342e+03, 4.7079375e-01, 1.7137401e-02, ..., 6.6197944e+00,
        0.0000000e+00, 5.6400002e+01]], dtype=float32)

1.0

In [ ]:
t1=model.transform(t1)
e1=model.transform(e1)
tt1=model.transform(tt1)